 
##Example 1 - Gold vs. Gold Mining##

*****

##Execute each cell by clicking on it and pressing shift + enter.

******

In [ ]:
miners = symbols('GDX') #Create a variable for our gold mining ETF
gold = symbols('GLD') #Create a variable for our gold ETF

data = get_pricing([miners, gold], #First we get the daily close prices for the two ETFs
            fields='close_price', 
            start_date='2006-06-01', #From 2007 through 2014
            end_date='2014-12-31')

data.plot() #We plot the prices together to see if there is an interesting correlation

#We also plot their relative returns. 

returns = data.pct_change().dropna()
returns.plot() 

****
Research is great for finding correlations among equities.The data above, plots `$GLD`, an ETF for the price of gold, against `$GDX`, an ETF looking at the gold mining industry.

****
To continue the analysis, we can import Seaborn, a statistical plotting library to use the distplot function. This allows us to plot the distribution of returns for `$GDX`
****

In [ ]:
import seaborn as sns

sns.distplot(returns.iloc[:,0:1]) 

************
We can also review the correlation between `$GLD` and `$GDX` by running a linear regression and plotting the best fitting line, the confidence interval (shaded regions) and the Pearson regression coefficient as well as p-value.
************

In [ ]:
#Get the historical close prices for USO and ITA individually

GLD = get_pricing(symbols(gold),
            fields='close_price',
            start_date='2007-01-02',
            end_date='2014-12-31')

GDX = get_pricing(symbols(miners),
            fields='close_price',
            start_date='2007-01-02',
            end_date='2014-12-31')

#Calculate the returns for each

returns_GLD = GLD.pct_change().dropna()
returns_GDX = GDX.pct_change().dropna()

#Plot using Seaborn's jointplot function

sns.jointplot(returns_GLD, returns_GDX, kind='reg', size=12)


*******

From these graphs, you can begin to analyze if there is a possible opportunity to invest in the correlation between the gold and gold miner industries. 

If you return to the first cell, you can try out different combinations of stocks to look for something more or less correlated. Perhaps `$AAPL` and `$HPQ` or `$PEP` and `$KO`? 

********
#Example 2 - US Housing vs. S&P  500

We can also import external data and compare it to historical market data.

In this case, we've taken a csv showing housing price data from Quandl (https://www.quandl.com/FRED/ASPNHSUS-Average-Sales-Price-for-New-Houses-Sold-in-the-United-States,) stored the data in the /data directory in Quantopian research.

**********

In [ ]:
import matplotlib.pyplot as pyplot

# Load the ave sales prices of US homes from the csv residing in your data directory
us_housing_prices = local_csv('FRED-ASPNHSUS.csv', date_column='Date')\
    .sort_index(ascending=True)
    
# Get the historical prices for SPY, the ETF for the S&P 500
spy_daily_prices = get_pricing(
    'SPY', 
    fields='close_price', 
    start_date='2002-01-01', 
    end_date = '2014-12-31', 
    frequency='daily',
)    

# Normalize the data for easy comparison
spy_daily_prices = spy_daily_prices/spy_daily_prices.iloc[0]
us_housing_prices = us_housing_prices/us_housing_prices.iloc[0]

# Plot them together with matplotlib
us_housing_prices.plot(label="US Average Housing Price")
spy_daily_prices.plot(label="S&P 500 (SPY ETF)")


***********

##Example 3 - Plotting Returns and Transactions 

With the `get_backtest` function, you have access to all of the data created by any backtest you run. This gives you unlimited flexibility in how you analyze your results. 

Here we plot the returns and trasactions of this algo: https://www.quantopian.com/posts/quantopian-tutorial-with-portfolio-rebalance-algorithm-lesson-2-universe-fetcher-and-schedule-function

**********

In [ ]:
# Load the backtest results into results
results = get_backtest('548f4f3d885aef09019e9c36')

# Plot the ending portfolio value
ending_portfolio_value = results.cumulative_performance.ending_portfolio_value 
ending_portfolio_value.plot()

# Add buy/sell markers to the second plot
transactions = results.transactions
buys = transactions[(transactions.amount > 0)] #get all the buys
sells = transactions[(transactions.amount < 0)] # and all the sells
pyplot.plot(buys.index, ending_portfolio_value.ix[buys.index],
         '^', markersize=10, color='m', label='Buys') #plot the buys to be pink up arrows that are pink
pyplot.plot(sells.index, ending_portfolio_value.ix[sells.index], #plot the sells to be black down arrows
         'v', markersize=10, color='k', label='Sells')

# Set figure metadata
pyplot.title("Ending Portfolio Value with Buys and Sells")
pyplot.ylabel('portfolio value in $')
pyplot.legend(loc=0)
pyplot.show()